# A1111 Extension Installer (UI)

Install **Automatic1111** extensions into your WebUI **data directory** so they persist across pods:

- Data root (WEBUI_ROOT): `/workspace/a1111` (default in this image)
- Extensions dir: `/workspace/a1111/extensions`

**How it works**
- One unified list of extensions; our recommendations are **pre-checked**.
- Add **extra Git URLs** (one per line) if you want more.
- Safe to re-run: existing repos are updated with `git pull`.

> After installing, use the **Restart WebUI** cell (or Settings → Reload UI) so extensions load.

In [ ]:
# First time? Uncomment, run once, then Kernel → Restart
# !pip install -q ipywidgets tqdm requests huggingface_hub
print("Downloader UI ready (install dependencies once if widgets are missing).")

In [ ]:
# --- Paths / settings ---
import os
from pathlib import Path

# Where A1111 expects user data (models, embeddings, *extensions*, etc.)
WEBUI_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
EXT_DIR = WEBUI_ROOT / "extensions"
EXT_DIR.mkdir(parents=True, exist_ok=True)

print("WEBUI_ROOT:", WEBUI_ROOT)
print("Extensions directory:", EXT_DIR)

# Unified list of extensions; 'recommended': true means the checkbox starts checked
EXTENSIONS = [
  {"name": "Aspect Ratio",            "url": "https://github.com/alemelis/sd-webui-ar",                               "recommended": True},
  {"name": "Tag Autocomplete",        "url": "https://github.com/DominikDoom/a1111-sd-webui-tagcomplete",             "recommended": True},
  {"name": "Images Browser",          "url": "https://github.com/AlUlkesh/stable-diffusion-webui-images-browser",     "recommended": True},
  {"name": "Config Presets",          "url": "https://github.com/Zyin055/Config-Presets",                            "recommended": True},
  {"name": "ADetailer",               "url": "https://github.com/Bing-su/adetailer",                                 "recommended": True},
  {"name": "Ultimate SD Upscale",     "url": "https://github.com/Coyote-A/ultimate-upscale-for-automatic1111",        "recommended": True},
  {"name": "ControlNet",              "url": "https://github.com/Mikubill/sd-webui-controlnet",                      "recommended": False},
  {"name": "Dynamic Prompts",         "url": "https://github.com/adieyal/sd-dynamic-prompts",                        "recommended": False},
  {"name": "Openpose Editor",         "url": "https://github.com/fkunn1326/openpose-editor",                         "recommended": False}
]

print("Extensions listed:", len(EXTENSIONS))

## UI: choose what to install
- Check/uncheck any extensions.
- Paste extra Git URLs (one per line).

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

# Build checkboxes for the unified list
ext_checkboxes = []
for item in EXTENSIONS:
    cb = W.Checkbox(description=item["name"], value=bool(item.get("recommended", False)))
    ext_checkboxes.append((cb, item))

# Extra repo URLs
extra_text = W.Textarea(
    value="",
    placeholder="Paste additional extension Git repo URLs here (one per line).",
    layout=W.Layout(width="100%", height="120px")
)

btn = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

display(Markdown("**Extensions:**"))
for cb, _ in ext_checkboxes:
    display(cb)
display(Markdown("**Extra URLs (one per line):**"))
display(extra_text)
display(btn, out)

## Installer logic
Clones new repos or updates existing ones in place.

In [ ]:
import subprocess
from urllib.parse import urlparse
from pathlib import Path

def run(cmd, cwd=None):
    print("$", " ".join(cmd))
    return subprocess.run(cmd, cwd=cwd, check=True)

def repo_basename(url: str) -> str:
    # https://github.com/owner/repo(.git) -> repo
    path = urlparse(url).path.rstrip("/")
    name = path.split("/")[-1]
    return name[:-4] if name.endswith(".git") else name

def install_or_update(url: str, base: Path) -> Path:
    name = repo_basename(url)
    dst = base / name
    if not dst.exists():
        print(f"\n[install] {name} from {url}")
        run(["git", "clone", "--depth", "1", url, str(dst)])
    else:
        print(f"\n[update] {name}")
        try:
            run(["git", "-C", str(dst), "reset", "--hard"]) 
            run(["git", "-C", str(dst), "pull", "--ff-only"]) 
        except subprocess.CalledProcessError:
            print("  (non-ff; attempting rebase)")
            run(["git", "-C", str(dst), "pull", "--rebase", "--autostash"]) 
    return dst

def perform_install(ext_pairs, extra_lines):
    installed, errors = [], []

    # 1) Selected from list
    for cb, item in ext_pairs:
        if not cb.value:
            continue
        try:
            installed.append(install_or_update(item["url"], EXT_DIR))
        except Exception as e:
            errors.append((item["name"], str(e)))

    # 2) Extra URLs
    for url in [u.strip() for u in extra_lines if u.strip()]:
        try:
            installed.append(install_or_update(url, EXT_DIR))
        except Exception as e:
            errors.append((url, str(e)))

    return installed, errors

def on_click(_):
    out.clear_output()
    with out:
        print("Installing… this can take a minute if repos are large.")
        lines = extra_text.value.splitlines()
        installed, errors = perform_install(ext_checkboxes, lines)

        print("\n=== Summary ===")
        for p in installed:
            print("✔", p)
        if errors:
            print("\nErrors:")
            for name, msg in errors:
                print("✖", name, "→", msg)
        print("\nIf extensions don't appear, run the **Restart WebUI** cell below and then refresh the WebUI.")

btn.on_click(on_click)
print("Installer logic loaded.")

## Restart WebUI (required after installing)
This restarts A1111 under **supervisord**. If the supervisor control socket is unavailable, it falls back to killing `launch.py` so Supervisor auto-restarts it.

In [ ]:
import time, subprocess, shlex

print("Attempting to restart the A1111 WebUI service via supervisord …")
try:
    cmd = "supervisorctl -c /etc/supervisord.conf restart a1111"
    out = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    if out.returncode == 0 and "a1111:" in (out.stdout or ""):
        print((out.stdout or out.stderr).strip() or "✔ Restarted via supervisord")
    else:
        print("Supervisorctl socket unavailable, forcing process restart…")
        subprocess.run("pkill -f 'launch.py'", shell=True)
        time.sleep(5)
        print("✔ WebUI process terminated, Supervisor will auto-restart it")
    print("Wait ~15 seconds, then reload your WebUI tab (port 7860).")
except Exception as e:
    print("Restart attempt failed:", e)
    print("Manual command:")
    print("supervisorctl -c /etc/supervisord.conf restart a1111")

## VerifyList what’s currently in the data extensions folder.

In [ ]:
from pathlib import Path
p = Path('/workspace/a1111/extensions')
items = sorted([x.name for x in p.iterdir()]) if p.exists() else []
print("Extensions found:")
for name in items:
    print("✔", name)
print("Total:", len(items))